# Image Processing with PyImageJ
Perform image processing using ImageJ. If the image size is large, there might be insufficient memory, leading to process crashes. Therefore, it is necessary to use a PC with sufficient memory. Additionally, when using Java, it is essential to specify in advance how much memory will be used.

Replace 100g in "scyjava.config.add_option('-Xmx100g')" with the amount of memory Java should have. Reserve some memory for your core operating system and other programs; a good rule of thumb is to allocate no more than 80% of your physical RAM to Java.


  plugins_dir = '/Applications/Fiji/Fiji.app/plugins'
  
ij = imagej.init('/Applications/Fiji/Fiji.app')
This also needs to be modified according to the environment and OS used. The plugins_dir specifies the location of Fiji's plugins, and ij specifies the location of Fiji. For more details, please refer to the following: https://py.imagej.net/en/latest/01-Starting-PyImageJ.html


In [ ]:
import os
# Get the path of the current directory
current_directory = os.getcwd()
# Get the path two levels up from the current directory
dir = os.path.dirname(current_directory)
print(dir)


import imagej
import scyjava
import os

# This specifies how much memory Java can use. If 60 GB is available, change '-Xmx100g' to '-Xmx60g'.
scyjava.config.add_option('-Xmx100g')

# This also needs to be modified. It varies depending on the environment and OS used. plugins_dir specifies the location of Fiji's plugins, and ij specifies the location of Fiji. For more details, please refer to the following. https://py.imagej.net/en/latest/01-Starting-PyImageJ.html
ij = imagej.init('/Applications/Fiji/Fiji.app')
plugins_dir = '/Applications/Fiji/Fiji.app/plugins'

scyjava.config.add_option(f'-Dplugins.dir={plugins_dir}')



To ensure the code runs correctly, execute the following, and if ImageJ's version is displayed, it is functioning properly:

In [ ]:
print(ij.getVersion())

The macro performs the following operations:

* Setting batch mode: Batch mode is enabled for faster processing.
* Loading image files: Multiple image files (seqFISH_*.tif) are loaded from a specific directory.
* Combining images: The loaded images are combined into a stack.
* Converting to a hyperstack: The stack is converted into a hyperstack, setting properties for channels, slices, frames, and pixels.
* Splitting channels and converting to 32-bit: Channels are split and each is converted to a 32-bit image.
* Loading flat field and dark images: Correction images are loaded.
* Correcting with dark images: Dark images are subtracted from each channel.
* Normalizing with flat field images: To avoid values of 0, 1 is added before dividing by the flat field image.
* Merging channels: Four channels are merged to create a new hyperstack.
* Correcting 3D drift: 3D drift is corrected.
* Saving: The processed image is saved in the specified directory.

In [ ]:
import os

macro = """
#@ String dir
#@ int cycle
#@ int Pos
#@ int zslices
#@ float pixel_width
#@ float voxel_depth


setBatchMode(true);
// Batch mode

for (i = 1; i < cycle+1; i++) {
	open(dir+"/0_raw_data/raw_data/seqFISH_"+Pos+"_"+IJ.pad(i, 3)+".tif");
	}
	
run("Concatenate...", "all_open title=Stack open");
run("Stack to Hyperstack...", "order=xyczt(default) channels=4 slices="+zslices+" frames="+cycle+" display=Composite");
run("Properties...", "channels=4 slices="+zslices+" frames="+cycle+" pixel_width="+pixel_width+" pixel_height="+pixel_width+" voxel_depth="+voxel_depth+"");
run("Conversions...", " ");
run("Split Channels");

selectImage("C1-Stack");
run("32-bit");
selectImage("C2-Stack");
run("32-bit");
selectImage("C3-Stack");
run("32-bit");
selectImage("C4-Stack");
run("32-bit");

//Open flat field image
open(dir+"/0_raw_data/flat_field_image/AB9-647.tif");
open(dir+"/0_raw_data/flat_field_image/Cou-405.tif");
open(dir+"/0_raw_data/flat_field_image/FL-488.tif");
open(dir+"/0_raw_data/flat_field_image/RB-561.tif");
// Open dark image
open(dir+"/0_raw_data/flat_field_image/DarkImage.tif");


// Subtract dark image from each channel.
imageCalculator("Subtract 32-bit stack", "C1-Stack","DarkImage.tif");
imageCalculator("Subtract 32-bit stack", "C2-Stack","DarkImage.tif");
imageCalculator("Subtract 32-bit stack", "C3-Stack","DarkImage.tif");
imageCalculator("Subtract 32-bit stack", "C4-Stack","DarkImage.tif");
selectWindow("DarkImage.tif");
close();

// There will be places where the value is 0, so add 1 to the entire image. The image is further divided by the normalized flat field image. In addition, 1 is added.
selectWindow("C1-Stack");
run("Add...", "value=1 stack");
imageCalculator("Divide 32-bit stack", "C1-Stack","AB9-647.tif");
run("16-bit");
run("Add...", "value=1 stack");
selectWindow("AB9-647.tif");
close();

selectWindow("C2-Stack");
run("Add...", "value=1 stack");
imageCalculator("Divide 32-bit stack", "C2-Stack","RB-561.tif");
run("16-bit");
run("Add...", "value=1 stack");
selectWindow("RB-561.tif");
close();

selectWindow("C3-Stack");
run("Add...", "value=1 stack");
imageCalculator("Divide 32-bit stack", "C3-Stack","FL-488.tif");
run("16-bit");
run("Add...", "value=1 stack");
selectWindow("FL-488.tif");
close();

selectWindow("C4-Stack");
run("Add...", "value=1 stack");
imageCalculator("Divide 32-bit stack", "C4-Stack","Cou-405.tif");
run("16-bit");
run("Add...", "value=1 stack");
// Only the Hoechst channel should be breach corrected and the brightness values should be adjusted.
run("Bleach Correction", "correction=[Simple Ratio] background=0");
selectWindow("C4-Stack");
close();
selectWindow("DUP_C4-Stack");
rename("C4-Stack");
selectWindow("Cou-405.tif");
close();



//Merge the 4 colors here, but set the Nucleus channel to 1.
run("Merge Channels...", "c1=C4-Stack c2=C1-Stack c3=C2-Stack c4=C3-Stack create ignore");

run("Stack to Hyperstack...", "order=xyczt(default) channels=4 slices="+zslices+" frames="+cycle+" display=Composite");
run("Properties...", "channels=4 slices="+zslices+" frames="+cycle+" pixel_width="+pixel_width+" pixel_height="+pixel_width+" voxel_depth="+voxel_depth+"");
rename("Composite");

// correct 3D drift
run("Correct 3D drift", "channel=1 edge_enhance only=0 lowest=1 highest=51 max_shift_x=50 max_shift_y=50 max_shift_z=30");
selectWindow("Composite");
close();
selectWindow("registered time points");
rename("Stack");


// Store each field of view separately.


for (i = 1; i < cycle+1; i++) {
	selectWindow("Stack");
	run("Duplicate...", "title=Stack duplicate frames="+i+"");
	rename("to_be_saved");
	saveAs("Tiff", dir+"/1_processed_images/1_FL_corrected/Pos"+IJ.pad(Pos, 2)+"/ND_"+IJ.pad(i, 2)+"");
	close();
}

selectWindow("Stack");
close("*");

run("Quit");
"""

# Repeat so that the value of Pos goes from 1 to 3
for Pos in range(1, 4):  
	Pos_padded = str(Pos).zfill(2)

	# Assemble directory paths
	directory_path = os.path.join(dir, "1_processed_images", "1_FL_corrected", "Pos" + Pos_padded)
	# Create a directory
	os.makedirs(directory_path, exist_ok=True)

	args = {
    'dir': dir,
    'cycle' : 83,
	'Pos' : Pos,
	'zslices' : 51,
	'pixel_width' : 0.1300000,
	'voxel_depth' : 0.2000000}

	ij.py.run_macro(macro, args)

Furthermore, the following macro conducts these operations:
* Setting batch mode: Batch mode is enabled for acceleration of processing.
* Loading an image sequence: An image sequence is loaded from a specific directory and converted into a hyperstack.
* Duplicating and measuring specific areas: Specific areas of the image are duplicated and measured.
* Selecting slices: The range of slices is determined based on the middle slice.
* Cropping the image: The image is cropped based on the determined slice range.
* Descriptor-based series registration: Descriptor-based registration is applied to the image series to improve alignment.
* Duplicating and measuring specific areas again: Specific areas of the image are duplicated and measured again.
* Minimum intensity Z-projection: A minimum intensity Z-projection is performed on the specified slices.
* Setting thresholds and particle analysis: After setting the threshold and converting to a mask, particle analysis is performed.
* Saving the image: The processed image is saved in TIFF format.
* Cleanup: Closes all open windows and exits ImageJ.

In [ ]:
macro = """
#@ String dir
#@ int cycle
#@ int Pos
#@ int zslices
#@ float pixel_width
#@ float voxel_depth

setBatchMode("hide");

File.openSequence(dir+"/1_processed_images/1_FL_corrected/Pos"+IJ.pad(Pos, 2)+"/");
getDimensions(width, height, channels, slices, frames);
z_slices = slices/(4*cycle);
run("Stack to Hyperstack...", "order=xyczt(default) channels=4 slices="+z_slices+" frames="+cycle+" display=Composite");

rename("stack");

makeRectangle(25, 19, 111, 95);
run("Duplicate...", "duplicate channels=1");
run("Set Measurements...", "mean integrated redirect=None decimal=5");
run("Measure Stack...", "channels slices frames order=czt(default)");

//https://forum.image.sc/t/results-to-array/8727/3
RawIntDen_lower = newArray(nResults);
RawIntDen_upper = newArray(nResults);
//index_N = newArray(nResults);
Slice_N_low = newArray(nResults); 
Slice_N_upper = newArray(nResults); 

getDimensions(width, height, channels, slices, frames); 
med_slice_N = round(slices/ 2);
//print (med_slice_N);

for(i=nResults-1; i>-1; i--) {
	//index_N[i] = i;
	RawIntDen_lower[i] = getResult("RawIntDen", i);
	RawIntDen_upper[i] = getResult("RawIntDen", i);
	Slice_N_low[i] = getResult("Slice", i);
	Slice_N_upper[i] = getResult("Slice", i);
	if ( RawIntDen_lower[i] != 0 ){RawIntDen_lower = Array.deleteIndex(RawIntDen_lower, i);
	 								Slice_N_low = Array.deleteIndex(Slice_N_low, i);}
	if ( RawIntDen_upper[i] != 0 ){RawIntDen_upper = Array.deleteIndex(RawIntDen_upper, i);
	 								Slice_N_upper = Array.deleteIndex(Slice_N_upper, i);}
}

print(Slice_N_low.length);

for(i=Slice_N_low.length-1; i>-1; i--) {
	if ( Slice_N_low[i] > med_slice_N ){Slice_N_low = Array.deleteIndex(Slice_N_low, i);}
}

for(i=Slice_N_upper.length-1; i>-1; i--) {
	if ( Slice_N_upper[i] < med_slice_N ){Slice_N_upper = Array.deleteIndex(Slice_N_upper, i);}
}

Array.getStatistics(Slice_N_upper, min, max, mean, stdDev);
maxz = min
Array.getStatistics(Slice_N_low, min, max, mean, stdDev);
minz = max
print("Save from z"+minz+1+" to z"+maxz-1+".");

selectWindow("stack-1");
close();

selectWindow("stack");
run("Select None");
run("Duplicate...", "title=stack-1 duplicate slices="+minz+1+"-"+maxz-1+"");

selectWindow("stack");
close();

selectWindow("stack-1");
getDimensions(width, height, channels, slices, frames); 
med_slice_N = round(slices/ 2);

run("Duplicate...", "duplicate channels=1 slices="+med_slice_N+"");
run("Properties...", "channels=1 slices="+cycle+" frames=1 pixel_width="+pixel_width+" pixel_height="+pixel_width+" voxel_depth="+voxel_depth+"");
run("Z Project...", "projection=[Min Intensity]");


setThreshold(1, 65535);

// Get image width and height
width = getWidth();
height = getHeight();

// Calculate center coordinates
x = width / 2;
y = height / 2;

// Apply wand tool to center coordinates
doWand(x, y);


run("Convert to Mask");
run("Analyze Particles...", "size=0-Infinity");
selectWindow("stack-1");
run("Restore Selection");

selectWindow("MIN_stack-1-1");
close();
selectWindow("stack-1-1");
close();
selectWindow("stack-1");
run("Crop");

run("Clear Results");


rename("stack");

run("Descriptor-based series registration (2d/3d + t)", "series_of_images=stack brightness_of=Low approximate_size=[5 px] type_of_detections=[Minima & Maxima] subpixel_localization=[3-dimensional quadratic fit] transformation_model=[Affine (3d)] images_are_roughly_aligned number_of_neighbors=3 redundancy=1 significance=3 allowed_error_for_ransac=5 global_optimization=[All against first image (no global optimization)] range=5 choose_registration_channel=1 image=[Fuse and display] interpolation=[Linear Interpolation]");

selectWindow("stack");
close();




rename("stack");

makeRectangle(25, 19, 111, 95);
run("Duplicate...", "duplicate channels=1");
run("Set Measurements...", "mean integrated redirect=None decimal=5");
run("Measure Stack...", "channels slices frames order=czt(default)");

//https://forum.image.sc/t/results-to-array/8727/3
RawIntDen_lower = newArray(nResults);
RawIntDen_upper = newArray(nResults);
//index_N = newArray(nResults);
Slice_N_low = newArray(nResults); 
Slice_N_upper = newArray(nResults); 

getDimensions(width, height, channels, slices, frames); 
med_slice_N = round(slices/ 2);
//print (med_slice_N);

for(i=nResults-1; i>-1; i--) {
	//index_N[i] = i;
	RawIntDen_lower[i] = getResult("RawIntDen", i);
	RawIntDen_upper[i] = getResult("RawIntDen", i);
	Slice_N_low[i] = getResult("Slice", i);
	Slice_N_upper[i] = getResult("Slice", i);
	if ( RawIntDen_lower[i] != 0 ){RawIntDen_lower = Array.deleteIndex(RawIntDen_lower, i);
	 								Slice_N_low = Array.deleteIndex(Slice_N_low, i);}
	if ( RawIntDen_upper[i] != 0 ){RawIntDen_upper = Array.deleteIndex(RawIntDen_upper, i);
	 								Slice_N_upper = Array.deleteIndex(Slice_N_upper, i);}
}

print(Slice_N_low.length);

for(i=Slice_N_low.length-1; i>-1; i--) {
	if ( Slice_N_low[i] > med_slice_N ){Slice_N_low = Array.deleteIndex(Slice_N_low, i);}
}

for(i=Slice_N_upper.length-1; i>-1; i--) {
	if ( Slice_N_upper[i] < med_slice_N ){Slice_N_upper = Array.deleteIndex(Slice_N_upper, i);}
}

Array.getStatistics(Slice_N_upper, min, max, mean, stdDev);
maxz = min
Array.getStatistics(Slice_N_low, min, max, mean, stdDev);
minz = max
print("Save from z"+minz+1+" to z"+maxz-1+".");

selectWindow("stack-1");
close();

selectWindow("stack");
run("Select None");
run("Duplicate...", "title=stack-1 duplicate slices="+minz+1+"-"+maxz-1+"");

selectWindow("stack");
close();

selectWindow("stack-1");
getDimensions(width, height, channels, slices, frames); 
med_slice_N = round(slices/ 2);

run("Duplicate...", "duplicate channels=1 slices="+med_slice_N+"");
run("Properties...", "channels=1 slices="+cycle+" frames=1 pixel_width="+pixel_width+" pixel_height="+pixel_width+" voxel_depth="+voxel_depth+"");
run("Z Project...", "projection=[Min Intensity]");


setThreshold(1, 65535);

// Get image width and height
width = getWidth();
height = getHeight();

// Calculate center coordinates
x = width / 2;
y = height / 2;

// Apply wand tool to center coordinates
doWand(x, y);


run("Convert to Mask");
run("Analyze Particles...", "size=0-Infinity");
selectWindow("stack-1");
run("Restore Selection");

selectWindow("MIN_stack-1-1");
close();
selectWindow("stack-1-1");
close();
selectWindow("stack-1");
run("Crop");

run("Clear Results");




saveAs("Tiff", dir+"/1_processed_images/2_DBR_corrected/Pos_"+IJ.pad(Pos, 2)+".tif");
close("*");

run("Quit");
"""

# Assemble the directory path
directory_path = os.path.join(dir, "1_processed_images","2_DBR_corrected")

# Make the directory if it doesn't exist already
os.makedirs(directory_path, exist_ok=True)

# Repeats with Pos values from 1 to 3
for Pos in range(1, 4):  
	
	args = {
    'dir': dir,
    'cycle' : 83,
	'Pos' : Pos,
	'zslices' : 51,
	'pixel_width' : 0.1300000,
	'voxel_depth' : 0.2000000}

	ij.py.run_macro(macro, args)
 

The following macro performs the following operations:
* Loading Image Files: Image files are loaded from a specific directory and opened under the name "original".  
<br>
* seq-RNA-FISH Processing:
  *  For frames from the first cycle to last_cycle_RNAseqFISH, the following operations are carried out for each channel (1 to 3):
      * Duplicate the relevant channel and frame from the "original" window, naming it "stack".
      * Convert this duplicate to grayscale.
      * Save the converted image to a specific directory in TIFF format. The file name is based on the frame and channel.
      * Close the processed image. <br> <br>

* seq-DNA-FISH Processing:
  *  For frames from the cycle following last_cycle_RNAseqFISH to last_cycle_DNAseqFISH, perform the same process as for RNA-FISH, but with different file names for saving.<br> <br>

* seq-IF-FISH Processing: <br>
  *  For frames from the cycle following last_cycle_DNAseqFISH to the end, execute the same steps as for RNA-FISH and DNA-FISH processing, but again with different file names for saving.
Close all windows: Once processing is complete, close all open windows.

In [ ]:
macro = """
#@ String dir
#@ int cycle
#@ int Pos
#@ int zslices
#@ float pixel_width
#@ float voxel_depth
#@ int last_cycle_RNAseqFISH
#@ int last_cycle_DNAseqFISH


    close("*");
	open(dir+"/1_processed_images/2_DBR_corrected/Pos_"+IJ.pad(Pos, 2)+".tif");
	
	rename("original");
	
	//RNA-FISH
	for (frames = 1; frames < (last_cycle_RNAseqFISH+1); frames++) {
		for (channels = 1; channels < 5; channels++) {
			selectWindow("original");
			
			run("Duplicate...", "title=stack duplicate channels="+channels+" frames="+frames+"");
			rename("stack");
			run("Grays");
	
			if (channels == 1) {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C4_TyR.tif");
			}else {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C"+(channels-1)+"_TyR.tif");
			}
			close();
		}
	}
	
	
	//DNA-FISH
	for (frames = last_cycle_RNAseqFISH+1; frames < (last_cycle_DNAseqFISH+1); frames++) {
		for (channels = 1; channels < 5; channels++) {
			selectWindow("original");
			
			run("Duplicate...", "title=stack duplicate channels="+channels+" frames="+frames+"");
			rename("stack");
			run("Grays");
	
			if (channels == 1) {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C4_TyD.tif");
			}else {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C"+(channels-1)+"_TyD.tif");
			}
			close();
		}
	}
	
	
	//seqIF
	for (frames = last_cycle_DNAseqFISH+1; frames < (cycle); frames++) {
		for (channels = 1; channels < 5; channels++) {
			selectWindow("original");
			
			run("Duplicate...", "title=stack duplicate channels="+channels+" frames="+frames+"");
			rename("stack");
			run("Grays");
	
	
			if (channels == 1) {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C4_TyI.tif");
			}else {
				saveAs("Tiff", dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Pos, 2)+"/ND"+IJ.pad(frames, 2)+"_C"+(channels-1)+"_TyI.tif");
			}
			close();
		}
	}

close("*");


run("Quit");

"""


# Repeat so that the value of Pos goes from 1 to 3
for Pos in range(1, 4):  
	Pos_padded = str(Pos).zfill(2)

	# Assemble directory paths
	directory_path = os.path.join(dir, "1_processed_images","3_diveded_files", "Pos" + Pos_padded)

	# Create a directory
	os.makedirs(directory_path, exist_ok=True)
 
	args = {
	'dir': dir,
	'cycle' : 83,
	'Pos' : Pos,
	'zslices' : 51,
	'pixel_width' : 0.1300000,
	'voxel_depth' : 0.2000000,
	'last_cycle_RNAseqFISH' : 31,
	'last_cycle_DNAseqFISH' : 74}
 
	ij.py.run_macro(macro, args)
 

The following macro executes the following operations:
* Setting Batch Mode: Batch mode is enabled to suppress updates to the user interface and improve processing speed.
<br><br>
* Loading and Formatting Images:  
  * Load the DNA-seqFISH image sequence from a specific position.
  * Obtain the dimensions of the loaded images and calculate the number of slices.
  * Convert the image stack into a hyperstack and display it in grayscale.
<br>
<br>
* Performing Average Intensity Projection:
  * Convert frames of the DNA-seqFISH image into slices and perform an average intensity projection.
  * This process highlights the fiducial marker areas.
  * Enhance the contrast. 
<br>
<br>
* Performing Maximum Intensity Projection:
  * Similarly, execute a maximum intensity projection on the DNA-seqFISH image.
  * This produces an overlay image of all bright spots.
  * Further enhance the contrast.
  * Close the original DNA-seqFISH image.  
<br>
<br>
* Converting Image Formats:
  * Convert the images from maximum and average intensity projections to a 32-bit format.  
<br>
<br>
* Saving Images:
  * Save the average intensity projection image as "Fiducial_enhance_stack.tif".
  * Save the maximum intensity projection image as "DNAseqFISH_foci_enhance_stack.tif".
  * Close all windows.

In [ ]:
macro = """
#@ String dir
#@ int cycle
#@ int Pos
#@ int zslices
#@ float pixel_width
#@ float voxel_depth


setBatchMode("hide");
for (Position = 1; Position < Pos+1; Position++) {

	//setBatchMode("hide");
	

	//Open seq-DNA-FISH image
	File.openSequence(dir+"/1_processed_images/3_diveded_files/Pos"+IJ.pad(Position, 2)+"/", " filter=_C3_TyD.tif start=2 count=40");
	rename("DNA_seqFISH");
	
	getDimensions(width, height, channels, slices, frames);
	z_slices = slices/(40);
 	
  run("Stack to Hyperstack...", "order=xyczt(default) channels=1 slices="+z_slices+" frames=40 display=Grayscale");


	//Convert t to z in seq-DNA-FISH images and perform average projection. Since all images have fiducial markers, this operation highlights the fiducial marker areas.
	selectWindow("DNA_seqFISH");
	run("Re-order Hyperstack ...", "channels=[Channels (c)] slices=[Frames (t)] frames=[Slices (z)]");
	run("Z Project...", "projection=[Average Intensity] all");
	run("Enhance Contrast", "saturated=0.01");


	//Further maximum intensity projection processing is performed on the seq-DNA-FISH image converted from t to z. This operation results in a superimposed image of all spots.
	selectWindow("DNA_seqFISH");
	run("Z Project...", "projection=[Max Intensity] all");
	run("Enhance Contrast", "saturated=0.35");
	selectWindow("DNA_seqFISH");
	close();


	run("Conversions...", " ");
	selectWindow("MAX_DNA_seqFISH");
	run("32-bit");
	selectWindow("AVG_DNA_seqFISH");
	run("32-bit");
	

	//setBatchMode("show");
	selectWindow("AVG_DNA_seqFISH");
	saveAs("Tiff", dir+"/1_processed_images/4_segmentation/Pos"+IJ.pad(Position, 2)+"/Fiducial_enhance_stack.tif");
	selectWindow("MAX_DNA_seqFISH");
	saveAs("Tiff", dir+"/1_processed_images/4_segmentation/Pos"+IJ.pad(Position, 2)+"/DNAseqFIS_foci_enhance_stack.tif");
	close("*");

}
"""

# Repeat so that the value of Pos goes from 1 to 3
for Pos in range(1, 4):  
	Pos_padded = str(Pos).zfill(2)

	# Assemble directory paths
	directory_path = os.path.join(dir, "1_processed_images","4_segmentation", "Pos" + Pos_padded)

	# Create a directory
	os.makedirs(directory_path, exist_ok=True)

	args = {
    'dir': dir,
    'cycle' : 83,
	'Pos' : Pos,
	'zslices' : 51,
	'pixel_width' : 0.1300000,
	'voxel_depth' : 0.2000000}

	ij.py.run_macro(macro, args)